<a href="https://colab.research.google.com/github/bellatchen/Assignments/blob/main/Abalone_PreProcessing_Retake_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bella Chen

In [26]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

In [27]:
# mounting Data
abalone = "/content/drive/MyDrive/Coding Dojo/Data Sets/abalone.data"
abalone_df_original = pd.read_csv(abalone)
abalone_df_original
abalone_df_1 = abalone_df_original.copy(deep= True)
print(abalone_df_1)

     Sex  Length  Diameter  Height  Whole Weight  Shucked Weight  \
0      M   0.455     0.365   0.095        0.5140          0.2245   
1      M   0.350     0.265   0.090        0.2255          0.0995   
2      F   0.530     0.420   0.135        0.6770          0.2565   
3      M   0.440     0.365   0.125        0.5160          0.2155   
4      I   0.330     0.255   0.080        0.2050          0.0895   
...   ..     ...       ...     ...           ...             ...   
4172   F   0.565     0.450   0.165        0.8870          0.3700   
4173   M   0.590     0.440   0.135        0.9660          0.4390   
4174   M   0.600     0.475   0.205        1.1760          0.5255   
4175   F   0.625     0.485   0.150        1.0945          0.5310   
4176   M   0.710     0.555   0.195        1.9485          0.9455   

      Viscera Weight  Shell Weight  Rings  
0             0.1010        0.1500     15  
1             0.0485        0.0700      7  
2             0.1415        0.2100      9  
3      

In [28]:
#Checking for Duplicates
abalone_df_1.duplicated().sum()

0

In [29]:
#check for missing data
abalone_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole Weight    4177 non-null   float64
 5   Shucked Weight  4177 non-null   float64
 6   Viscera Weight  4177 non-null   float64
 7   Shell Weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [30]:
#checking values and counts
abalone_df_1["Sex"].value_counts()
#everything is uniform

M    1528
I    1342
F    1307
Name: Sex, dtype: int64

In [31]:
# Separate data into features matrix (X) and target vector (y)
#defining X and y; predicting charges
X = abalone_df_1.drop(columns=["Rings"])
y = abalone_df_1["Rings"]
# Train/test split the data. Please use the random number 42 for consistency.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [32]:
 #verifying split
 len(X_train)

3132

Tasks:

3) Use column transformers to transform the appropriate columns in the appropriate ways.

For the Column Transformations:

    a) Select the categorical columns and the numerical columns

    b) Use a OneHotEncoder to encode the categorical columns

    c) Use a StandardScaler to scale the numeric columns

    d) Use a ColumnTransformer to match the transformation to the type of column

    e) Transform the data and display the resulting Numpy array.
  

Categories are: 

Nominal: 
- Sex

Numeric:
- Length
- Diameter
- Height
- Whole Weight
- Shucked Weight
- Viscera Weight
- Shell Weight

In [33]:
#use column transformer instead
#make categorical + numeric selector
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
scaler = StandardScaler()

#create tuples - can make as many as needed
num_tuple = (scaler, num_selector)
cat_tuple = (ohe_encoder, cat_selector)

col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder = "passthrough")
col_transformer.fit(X_train)

X_train_processed =  col_transformer.transform(X_train)
X_test_processed = col_transformer.transform(X_test)

X_train_processed

array([[ 0.74929076,  0.46422584, -0.11886923, ...,  1.        ,
         0.        ,  0.        ],
       [-0.09025371, -0.14465442, -0.0016468 , ...,  1.        ,
         0.        ,  0.        ],
       [ 1.12708577,  1.22532616,  0.81891021, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.13223093, -0.14465442, -0.35331409, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.41347297,  0.56570588, -0.47053652, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.58138187,  0.66718592,  0.46724292, ...,  1.        ,
         0.        ,  0.        ]])

In [38]:
#pulling column names from numerical columns
final_features = num_selector(X_train)
final_features

['Length',
 'Diameter',
 'Height',
 'Whole Weight',
 'Shucked Weight',
 'Viscera Weight',
 'Shell Weight']

In [39]:
# pulling column names from categorical columns
cat_features = list(col_transformer.named_transformers_['onehotencoder'].get_feature_names(cat_selector(X_train)))
cat_features

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['Sex_F', 'Sex_I', 'Sex_M']

In [40]:
#combining all column names from numerical and categorical columns into one 
final_features.extend(cat_features)

In [41]:
#showing all column names
final_features

['Length',
 'Diameter',
 'Height',
 'Whole Weight',
 'Shucked Weight',
 'Viscera Weight',
 'Shell Weight',
 'Sex_F',
 'Sex_I',
 'Sex_M']

In [42]:
#Dataframe with column names
df = pd.DataFrame(X_train_processed,columns = final_features)
df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Sex_F,Sex_I,Sex_M
0,0.749291,0.464226,-0.118869,0.457447,0.499098,0.743973,0.241135,1.0,0.0,0.0
1,-0.090254,-0.144654,-0.001647,-0.301655,-0.364269,-0.514040,-0.145838,1.0,0.0,0.0
2,1.127086,1.225326,0.818910,1.523852,1.692114,1.544526,1.179902,0.0,0.0,1.0
3,-0.593980,-0.449095,-1.056649,-0.651696,-0.617673,-0.738195,-0.647469,0.0,0.0,1.0
4,-0.258163,-0.093914,0.350020,-0.052352,-0.572823,-0.605532,0.785763,0.0,0.0,1.0
